In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns # 0.11.2
from PIL import Image 
import cv2 # apt-get install libgl1-mesa-glx 설치로 cv2 문제 해결
import os
from torchvision import transforms
data_dir = os.listdir("../input/data")

In [3]:
# Train Dataset 경로
train_dir = '/opt/ml/input/data/train'

# meta 데이터와 이미지 경로를 불러옵니다. (테스트 데이터의 )
train_df = pd.read_csv(os.path.join(train_dir, 'train.csv'))
image_dir = os.path.join(train_dir, 'images')

image_paths = [os.path.join(image_dir, img_path) for img_path in train_df.path]

## no_mask & incorrect
- no_mask & Incorrect mask 증가

In [ ]:
def transform(img, idx):
    if idx == 0:
        torchvision_transform = transforms.Compose([
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.1, hue=0.1)])
    elif idx == 1:
        torchvision_transform = transforms.Compose([transforms.RandomHorizontalFlip(p = 1)])
    else:
        torchvision_transform = transforms.Compose([
        transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.2, hue=0.2)])
    return torchvision_transform(img)

target_ls = ["normal.jpg", "incorrect_mask.jpg"]
for path in image_paths:
    picture_list = os.listdir(path)
    picture_list = [fname for fname in picture_list if fname[0] != "."] # 임시파일 제외
    for j in picture_list:
        if j in target_ls:
            target_img_path = os.path.join(path, j)
            img = Image.open(target_img_path).convert("RGB") # 이미지 불러오기
            for k in range(3):
                img = transform(img, k)
                tag = j.split('.')[0]
                print(f'{path}/{tag}{k}.jpg')
#                 img.save(f'{path}/{tag}{k}.jpg')

## age_old
- 60세 이상 2배로 데이터 증강

In [ ]:
# Train Dataset 경로
train_dir = '/opt/ml/input/data/train'

# meta 데이터와 이미지 경로를 불러옵니다. (테스트 데이터의 )
train_df = pd.read_csv(os.path.join(train_dir, 'train.csv'))
image_dir = os.path.join(train_dir, 'images')

age_to_old = ['004348']
for i in age_to_old:
    train_df.loc[train_df['id'] == i,'age'] = 60 # middle -> old

# image_paths = [os.path.join(image_dir, img_path) for img_path in train_df.path]

In [26]:
# train_df 나이 변경 (from dataset)
age_60_ls = train_df.loc[train_df.age== 60,'path']
image_paths = [os.path.join(image_dir, img_path) for img_path in age_60_ls]

In [49]:
len(image_paths)

192

In [50]:
def transform(img):
    torchvision_transform = transforms.Compose([
        transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.2),
        transforms.RandomHorizontalFlip(p=1)])
    return torchvision_transform(img)

In [ ]:
import time
time0 = time.time()           # 코드 시작 시간: time0
for path in image_paths:
    picture_list = os.listdir(path)
    picture_list = [fname for fname in picture_list if fname[0] != "."] # 임시파일 제외
#     fig, ax = plt.subplots(1, len(picture_list), figsize=(20, 5))
    for i, j in enumerate(picture_list):
        target_img_path = os.path.join(path, j)
        img = Image.open(target_img_path).convert("RGB") # 이미지 불러오기
        img = transform(img)
        tag = j.split('.')[0]
        img.save(f'{path}/add_{tag}.jpg')
#         print(f'{path}/add_{tag}.jpg')
#         ax[i].imshow(img)
time.strftime('%M분 %S초 소요', time.localtime(time.time()-time0))